In [22]:
from __future__ import unicode_literals

import pandas as pd
import numpy as np
from sklearn import preprocessing
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

# 1. Preparing data - 28 weeks of life of the banana plant 
---

We load the different datasets which compose the  time series from January 12th to August 03rd 

## 1.1. Loading phreatic level  dataset - From January 12th to April 12th

In [23]:
phreatic_level_Jan12_Apr12 = pd.read_csv('../../../../data/raw/FincaPorvenir/Drenajes/DatosNivelFreatico/28-weeks_January-12_August-03_2018/' \
                                 'From_2018-01-12 00_00_00_To_2018-04-12 23_59_59.csv', )

In [24]:
print("THE DIMENSIONALITY IS: " +'\n')
print(phreatic_level_Jan12_Apr12.shape)
# Numero de columnas
print("Columns number", len(phreatic_level_Jan12_Apr12.columns))
# Numero de registros de la columna PORVL10N1
print("PORVL10N1 column rows number",len(phreatic_level_Jan12_Apr12.PORVL10N1))
phreatic_level_Jan12_Apr12.head()

THE DIMENSIONALITY IS: 

(2149, 76)
Columns number 76
PORVL10N1 column rows number 2149


,Fecha,Hora,PORVL2N1,Fecha.1,Hora.1,PORVL2N2,Fecha.2,Hora.2,PORVL4N1,Fecha.3,...,PORVL21N4,Fecha.22,Hora.22,PORVL21N5,Fecha.23,Hora.23,PORVL24N1,Fecha.24,Hora.24,PORVL24N2
0,2018-01-12,01:37:47,0.65,2018-01-12,01:58:22,0.71,2018-01-12,01:28:49,0.96,2018-01-12,...,1.01,2018-01-12,01:33:02,0.96,2018-01-12,01:23:18,0.86,2018-01-12,01:07:28,1.31
1,2018-01-12,02:37:45,0.65,2018-01-12,02:58:22,0.71,2018-01-12,02:28:49,0.96,2018-01-12,...,1.01,2018-01-12,02:33:04,0.97,2018-01-12,02:23:17,0.85,2018-01-12,02:07:28,1.31
2,2018-01-12,03:37:50,0.64,2018-01-12,03:58:23,0.71,2018-01-12,03:28:50,0.96,2018-01-12,...,1.01,2018-01-12,03:33:05,0.94,2018-01-12,03:23:18,0.97,2018-01-12,03:07:29,1.31
3,2018-01-12,04:37:44,0.63,2018-01-12,04:58:23,0.71,2018-01-12,04:28:52,0.95,2018-01-12,...,1.01,2018-01-12,04:33:05,0.93,2018-01-12,04:23:19,0.96,2018-01-12,04:07:27,1.31
4,2018-01-12,05:37:45,0.61,2018-01-12,05:58:24,0.71,2018-01-12,05:28:48,0.94,2018-01-12,...,1.01,2018-01-12,05:33:06,0.92,2018-01-12,05:23:19,0.95,2018-01-12,05:07:27,1.31


## 1.2. Selecting  relevant index columns features
---

In [25]:
# We capture just the columns of the phreatic_level_Jan12_Apr12 dataframe 
# cols = list(nf_enero12_abril12.columns)
# print("The columns are: " + "\n\n", cols)

# We iterate by the columns list cols to get only of them that start with the prefix
# PORVL. We enumerate them and assing to pos_col variable
# pos_cols = [i for i, word  in enumerate(cols) if word.startswith('PORVL')]
# print(pos_cols)

Since the dataset has a columns called **`Fecha:`** and **`Hora:`** to each lot, which is not a numerical 

values, it will be removed so that it does not interfere **with our subsequent scaling**, 

so we are only going to reference the phreatic level values or samples of the ** `PORVLxNx` ** columns and 

assigning them to the dataframe `phreatic_level_Jan12_Apr12_lots_nodes` created such as follow:

- We use the [filter](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.filter.html) method to extract only the colums subset which names start with the ** `PORVLN` ** criteria 
 

In [26]:
phreatic_level_Jan12_Apr12_lots_nodes = phreatic_level_Jan12_Apr12.filter(regex=('PORVL.*'))
phreatic_level_Jan12_Apr12_lots_nodes.head()

,PORVL2N1,PORVL2N2,PORVL4N1,PORVL5N1,PORVL6N1,PORVL7N1,PORVL8N1,PORVL9N1,PORVL10N1,PORVL13N1,...,PORVL18N2,PORVL18N3,PORVL18N4,PORVL21N1,PORVL21N2,PORVL21N3,PORVL21N4,PORVL21N5,PORVL24N1,PORVL24N2
0,0.65,0.71,0.96,0.62,0.90,1.16,1.19,1.16,1.05,1.14,...,0.84,1.27,1.08,1.55,1.25,1.01,1.01,0.96,0.86,1.31
1,0.65,0.71,0.96,0.63,0.89,1.16,1.19,1.16,1.04,1.14,...,0.84,1.27,1.08,1.54,1.25,1.02,1.01,0.97,0.85,1.31
2,0.64,0.71,0.96,0.62,0.89,1.16,1.18,1.16,1.04,1.13,...,0.84,1.27,1.08,1.54,1.25,1.02,1.01,0.94,0.97,1.31
3,0.63,0.71,0.95,0.61,0.89,1.16,1.17,1.15,1.03,1.13,...,0.85,1.27,1.08,1.54,1.24,1.02,1.01,0.93,0.96,1.31
4,0.61,0.71,0.94,0.60,0.89,1.16,1.17,1.16,1.03,1.12,...,0.85,1.27,1.08,1.54,1.24,1.02,1.01,0.92,0.95,1.31


## 1.3 Filling **`NaN:`** values
---

We evaluate if `phreatic_level_Jan12_Apr12_lots_nodes` dataframe has null type `NaN`

Have null values.

In [27]:
print(phreatic_level_Jan12_Apr12_lots_nodes.isnull().any())
phreatic_level_Jan12_Apr12_lots_nodes.isnull().values.any()

PORVL2N1      True
PORVL2N2      True
PORVL4N1      True
PORVL5N1      True
PORVL6N1      True
PORVL7N1      True
PORVL8N1      True
PORVL9N1      True
PORVL10N1     True
PORVL13N1     True
PORVL14N1     True
PORVL15N1     True
PORVL16N1     True
PORVL16N2     True
PORVL18N1     True
PORVL18N2     True
PORVL18N3     True
PORVL18N4     True
PORVL21N1     True
PORVL21N2     True
PORVL21N3     True
PORVL21N4    False
PORVL21N5     True
PORVL24N1     True
PORVL24N2     True
dtype: bool


True

### What do we do with these missing values?

Some ideas:

- The first possibility is to remove the lines of the samples or records where there are missing data, but depending on the context or domain of the problem, this can be dangerous because it can contain crucial information. It is better to think of another idea

- A more common solution alternative is to take the average of the column values where the data is missing, in this case find the average of all the columns that our dataset has `phreatic_level_Jan12_Apr12_lots_nodes`

---
Other strategies are to take the median of the remaining values of the column where a value is missing, or take the most frequent value, which may be a good idea depending on the context

**Solution choosed**

We will take the average of the columns, replace the missing data here by the average of all the values in each column of the dataset `phreatic_level_Jan12_Apr12_lots_nodes` where there is null data.


Using [Imputer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html "sklearn.preprocessing.Imputer"),`scikit-learn` pre-processing data (one of many), we going to fill the dataset missing values:

[Imputer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html "sklearn.preprocessing.Imputer") have the following parameters:

![alt text](https://camo.githubusercontent.com/89c8f482070ff06c5ed62d807139d89a2353d9ca/68747470733a2f2f636c6475702e636f6d2f4e6b57416343684b736a2d3330303078333030302e706e67 "sklearn.preprocessing.Imputer")

As we said before, we will use as a strategy, fill in the missing data with the average of the other values in the column where data is missing.

`axis` is the axis along which we are going to do the operation of` Imputer`, by default it is equal to 0, which means that this operation will be along the columns



In [28]:
imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)

# We adjust the values transforming the columns where you need data 
# with the fit() method. 
# We pass the dataframe phreatic_level_Jan12_Apr12_lots_nodes and 
# with the variables where I can find the empty values, in this case 
# all variables [::] and fit 
imputer = imputer.fit(phreatic_level_Jan12_Apr12_lots_nodes[::])
phreatic_level_Jan12_Apr12_lots_nodes = imputer.transform(phreatic_level_Jan12_Apr12_lots_nodes[::])

The previous step generate the ** `phreatic_level_Jan12_Apr12_lots_nodes` ** array which don't have null values

In [29]:
phreatic_level_Jan12_Apr12_lots_nodes

array([[0.65      , 0.71      , 0.96      , ..., 0.96      , 0.86      ,
        1.31      ],
       [0.65      , 0.71      , 0.96      , ..., 0.97      , 0.85      ,
        1.31      ],
       [0.64      , 0.71      , 0.96      , ..., 0.94      , 0.97      ,
        1.31      ],
       ...,
       [0.9858562 , 1.23405136, 1.31666199, ..., 1.36848315, 1.31913146,
        1.57120188],
       [0.9858562 , 1.23405136, 1.31666199, ..., 1.36848315, 1.31913146,
        1.57120188],
       [0.9858562 , 1.23405136, 1.31666199, ..., 1.36848315, 1.31913146,
        1.57120188]])

In [30]:
print(phreatic_level_Jan12_Apr12_lots_nodes.shape)
np.isnan(phreatic_level_Jan12_Apr12_lots_nodes)

(2149, 25)


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

- Turn the ** `phreatic_level_Jan12_Apr12_lots_nodes` ** array to dataframe

In [31]:
# We check the column names dataset 
phreatic_level_Jan12_Apr12.columns.get_values()[:]

array(['Fecha', 'Hora', 'PORVL2N1', 'Fecha.1', 'Hora.1', 'PORVL2N2',
       'Fecha.2', 'Hora.2', 'PORVL4N1', 'Fecha.3', 'Hora.3', 'PORVL5N1',
       'Fecha.4', 'Hora.4', 'PORVL6N1', 'Fecha.5', 'Hora.5', 'PORVL7N1',
       'Fecha.6', 'Hora.6', 'PORVL8N1', 'Fecha.7', 'Hora.7', 'PORVL9N1',
       'Fecha.8', 'Hora.8', 'PORVL10N1', 'Fecha.9', 'Hora.9', 'PORVL13N1',
       'Fecha.10', 'Hora.10', 'PORVL14N1', 'Fecha.11', 'Hora.11',
       'PORVL15N1', 'Fecha.12', 'Hora.12', 'PORVL16N1', 'Fecha.13',
       'Hora.13', 'PORVL16N2', 'Fecha.14', 'Hora.14', 'PORVL18N1',
       'Fecha.15', 'Hora.15', 'PORVL18N2', 'Fecha.16', 'Hora.16',
       'PORVL18N3', 'Unnamed: 51', 'Fecha.17', 'Hora.17', 'PORVL18N4',
       'Fecha.18', 'Hora.18', 'PORVL21N1', 'Fecha.19', 'Hora.19',
       'PORVL21N2', 'Fecha.20', 'Hora.20', 'PORVL21N3', 'Fecha.21',
       'Hora.21', 'PORVL21N4', 'Fecha.22', 'Hora.22', 'PORVL21N5',
       'Fecha.23', 'Hora.23', 'PORVL24N1', 'Fecha.24', 'Hora.24',
       'PORVL24N2'], dtype=objec

In [32]:
# We refer the columns of interest into col list, to apply the new dataframe
# which are in where there are the phreatic level values
cols =  ['PORVL2N1', 'PORVL2N2', 'PORVL4N1', 'PORVL5N1', 'PORVL6N1', 'PORVL7N1', 'PORVL8N1', 'PORVL9N1', 'PORVL10N1', 
         'PORVL13N1', 'PORVL14N1', 'PORVL15N1', 'PORVL16N1', 'PORVL16N2', 'PORVL18N1', 'PORVL18N2', 'PORVL18N3', 
         'PORVL18N4', 'PORVL21N1', 'PORVL21N2', 'PORVL21N3', 'PORVL21N4', 'PORVL21N5', 'PORVL24N1', 'PORVL24N2']
# index = ['Row'+ str(i) for i in range(1, len(numpy_nivel_freatico)+1)]
# El automaticamente toma el indice para los registros que es cada numero de registro
phreatic_level_Jan12_Apr12_lots_nodes_df = pd.DataFrame(data=phreatic_level_Jan12_Apr12_lots_nodes, columns=cols)

In [33]:
phreatic_level_Jan12_Apr12_lots_nodes_df.head()

print("Don't have null values \ ", phreatic_level_Jan12_Apr12_lots_nodes_df.isnull().values.any())
phreatic_level_Jan12_Apr12_lots_nodes_df.isnull().values.any()


Don't have null values \  False


False

In [34]:
phreatic_level_Jan12_Apr12_lots_nodes_df.head()

,PORVL2N1,PORVL2N2,PORVL4N1,PORVL5N1,PORVL6N1,PORVL7N1,PORVL8N1,PORVL9N1,PORVL10N1,PORVL13N1,...,PORVL18N2,PORVL18N3,PORVL18N4,PORVL21N1,PORVL21N2,PORVL21N3,PORVL21N4,PORVL21N5,PORVL24N1,PORVL24N2
0,0.65,0.71,0.96,0.62,0.90,1.16,1.19,1.16,1.05,1.14,...,0.84,1.27,1.08,1.55,1.25,1.01,1.01,0.96,0.86,1.31
1,0.65,0.71,0.96,0.63,0.89,1.16,1.19,1.16,1.04,1.14,...,0.84,1.27,1.08,1.54,1.25,1.02,1.01,0.97,0.85,1.31
2,0.64,0.71,0.96,0.62,0.89,1.16,1.18,1.16,1.04,1.13,...,0.84,1.27,1.08,1.54,1.25,1.02,1.01,0.94,0.97,1.31
3,0.63,0.71,0.95,0.61,0.89,1.16,1.17,1.15,1.03,1.13,...,0.85,1.27,1.08,1.54,1.24,1.02,1.01,0.93,0.96,1.31
4,0.61,0.71,0.94,0.60,0.89,1.16,1.17,1.16,1.03,1.12,...,0.85,1.27,1.08,1.54,1.24,1.02,1.01,0.92,0.95,1.31


## 1.2 Loading 13 de abril al 03 de agosto

In [35]:
phreatic_level_Apr13_Aug03 = pd.read_csv('../../../../data/raw/FincaPorvenir/Drenajes//DatosNivelFreatico/28-weeks_January-12_August-03_2018/' \
                                 'From_2018-04-13 00_00_00_To_2018-08-03 23_59_59.csv', )

In [36]:
print(phreatic_level_Apr13_Aug03.shape)
# Numero de columnas
print("Columns number", len(phreatic_level_Apr13_Aug03.columns))
# Numero de registros de la columna PORVL10N1
print("PORVL10N1 column rows number",len(phreatic_level_Apr13_Aug03.PORVL10N1))
phreatic_level_Apr13_Aug03.head()

(2731, 76)
Columns number 76
PORVL10N1 column rows number 2731


,Fecha,Hora,PORVL2N1,Fecha.1,Hora.1,PORVL2N2,Fecha.2,Hora.2,PORVL4N1,Fecha.3,...,PORVL21N4,Fecha.22,Hora.22,PORVL21N5,Fecha.23,Hora.23,PORVL24N1,Fecha.24,Hora.24,PORVL24N2
0,2018-04-13,01:01:31,1.32,2018-04-13,01:13:01,1.27,2018-04-13,01:34:10,1.46,2018-04-13,...,1.7,2018-04-13,01:46:49,1.7,2018-04-13,01:15:52,1.47,2018-04-13,01:19:31,1.7
1,2018-04-13,02:01:30,1.32,2018-04-13,02:12:59,1.28,2018-04-13,02:34:10,1.46,2018-04-13,...,1.7,2018-04-13,02:46:52,1.7,2018-04-13,02:15:52,1.47,2018-04-13,02:19:32,1.7
2,2018-04-13,03:01:30,1.33,2018-04-13,03:13:00,1.28,2018-04-13,03:34:08,1.46,2018-04-13,...,1.7,2018-04-13,03:46:52,1.7,2018-04-13,03:15:52,1.47,2018-04-13,03:19:31,1.7
3,2018-04-13,04:01:30,1.33,2018-04-13,04:12:58,1.28,2018-04-13,04:34:08,1.46,2018-04-13,...,1.7,2018-04-13,04:46:54,1.7,2018-04-13,04:15:53,1.47,2018-04-13,04:19:31,1.7
4,2018-04-13,05:01:31,1.33,2018-04-13,05:13:01,1.28,2018-04-13,05:34:11,1.46,2018-04-13,...,1.7,2018-04-13,05:46:53,1.7,2018-04-13,05:15:52,1.47,2018-04-13,05:19:32,1.7


In [37]:
phreatic_level_Apr13_Aug03_lots_nodes = phreatic_level_Apr13_Aug03.filter(regex=('PORVL.*'))

In [38]:
phreatic_level_Apr13_Aug03_lots_nodes.head()

,PORVL2N1,PORVL2N2,PORVL4N1,PORVL5N1,PORVL6N1,PORVL7N1,PORVL8N1,PORVL9N1,PORVL10N1,PORVL13N1,...,PORVL18N2,PORVL18N3,PORVL18N4,PORVL21N1,PORVL21N2,PORVL21N3,PORVL21N4,PORVL21N5,PORVL24N1,PORVL24N2
0,1.32,1.27,1.46,1.55,1.68,1.7,1.67,1.7,1.69,1.7,...,1.7,1.7,1.7,1.61,1.48,1.7,1.7,1.7,1.47,1.7
1,1.32,1.28,1.46,1.56,1.68,1.7,1.67,1.7,1.70,1.7,...,1.7,1.7,1.7,1.63,1.48,1.7,1.7,1.7,1.47,1.7
2,1.33,1.28,1.46,1.56,1.68,1.7,1.67,1.7,1.70,1.7,...,1.7,1.7,1.7,1.63,1.47,1.7,1.7,1.7,1.47,1.7
3,1.33,1.28,1.46,1.56,1.68,1.7,1.67,1.7,1.70,1.7,...,1.7,1.7,1.7,1.62,1.47,1.7,1.7,1.7,1.47,1.7
4,1.33,1.28,1.46,1.57,1.68,1.7,1.67,1.7,1.70,1.7,...,1.7,1.7,1.7,1.62,1.47,1.7,1.7,1.7,1.47,1.7


In [39]:
imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
imputer = imputer.fit(phreatic_level_Apr13_Aug03_lots_nodes[::])
phreatic_level_Apr13_Aug03_lots_nodes = imputer.transform(phreatic_level_Apr13_Aug03_lots_nodes[::])
phreatic_level_Apr13_Aug03_lots_nodes

array([[1.32      , 1.27      , 1.46      , ..., 1.7       , 1.47      ,
        1.7       ],
       [1.32      , 1.28      , 1.46      , ..., 1.7       , 1.47      ,
        1.7       ],
       [1.33      , 1.28      , 1.46      , ..., 1.7       , 1.47      ,
        1.7       ],
       ...,
       [0.82069245, 0.80241544, 0.94910767, ..., 1.19825056, 0.9261183 ,
        1.16      ],
       [0.82069245, 0.80241544, 0.94910767, ..., 1.19825056, 0.9261183 ,
        1.17      ],
       [0.82069245, 0.80241544, 0.94910767, ..., 1.19825056, 0.9261183 ,
        1.27451648]])

In [40]:
np.isnan(phreatic_level_Apr13_Aug03_lots_nodes)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [41]:
# We refer the columns of interest into col list, to apply the new dataframe
# which are in where there are the phreatic level values
cols =  ['PORVL2N1', 'PORVL2N2', 'PORVL4N1', 'PORVL5N1', 'PORVL6N1', 'PORVL7N1', 'PORVL8N1', 'PORVL9N1', 'PORVL10N1', 
         'PORVL13N1', 'PORVL14N1', 'PORVL15N1', 'PORVL16N1', 'PORVL16N2', 'PORVL18N1', 'PORVL18N2', 'PORVL18N3', 
         'PORVL18N4', 'PORVL21N1', 'PORVL21N2', 'PORVL21N3', 'PORVL21N4', 'PORVL21N5', 'PORVL24N1', 'PORVL24N2']
phreatic_level_Apr13_Aug03_lots_nodes_df = pd.DataFrame(data=phreatic_level_Apr13_Aug03_lots_nodes, columns=cols)
phreatic_level_Apr13_Aug03_lots_nodes_df.isnull().values.any()

False

In [42]:
phreatic_level_Apr13_Aug03_lots_nodes_df.head()

,PORVL2N1,PORVL2N2,PORVL4N1,PORVL5N1,PORVL6N1,PORVL7N1,PORVL8N1,PORVL9N1,PORVL10N1,PORVL13N1,...,PORVL18N2,PORVL18N3,PORVL18N4,PORVL21N1,PORVL21N2,PORVL21N3,PORVL21N4,PORVL21N5,PORVL24N1,PORVL24N2
0,1.32,1.27,1.46,1.55,1.68,1.7,1.67,1.7,1.69,1.7,...,1.7,1.7,1.7,1.61,1.48,1.7,1.7,1.7,1.47,1.7
1,1.32,1.28,1.46,1.56,1.68,1.7,1.67,1.7,1.70,1.7,...,1.7,1.7,1.7,1.63,1.48,1.7,1.7,1.7,1.47,1.7
2,1.33,1.28,1.46,1.56,1.68,1.7,1.67,1.7,1.70,1.7,...,1.7,1.7,1.7,1.63,1.47,1.7,1.7,1.7,1.47,1.7
3,1.33,1.28,1.46,1.56,1.68,1.7,1.67,1.7,1.70,1.7,...,1.7,1.7,1.7,1.62,1.47,1.7,1.7,1.7,1.47,1.7
4,1.33,1.28,1.46,1.57,1.68,1.7,1.67,1.7,1.70,1.7,...,1.7,1.7,1.7,1.62,1.47,1.7,1.7,1.7,1.47,1.7


## 1.3 Forming the whole phreatic level dataset - January 12th to August 3rd 

We have the following dataframes:

`phreatic_level_Jan12_Apr12_lots_nodes_df` and `phreatic_level_Apr13_Aug03_lots_nodes_df`

Together match with shape (columns and rows).
We can concatenate them of this way:

In [43]:
phreatic_level_lots_nodes_df = pd.concat([phreatic_level_Jan12_Apr12_lots_nodes_df, phreatic_level_Apr13_Aug03_lots_nodes_df])
phreatic_level_lots_nodes_df.shape

(4880, 25)

In [44]:
phreatic_level_lots_nodes_df.head()

,PORVL2N1,PORVL2N2,PORVL4N1,PORVL5N1,PORVL6N1,PORVL7N1,PORVL8N1,PORVL9N1,PORVL10N1,PORVL13N1,...,PORVL18N2,PORVL18N3,PORVL18N4,PORVL21N1,PORVL21N2,PORVL21N3,PORVL21N4,PORVL21N5,PORVL24N1,PORVL24N2
0,0.65,0.71,0.96,0.62,0.90,1.16,1.19,1.16,1.05,1.14,...,0.84,1.27,1.08,1.55,1.25,1.01,1.01,0.96,0.86,1.31
1,0.65,0.71,0.96,0.63,0.89,1.16,1.19,1.16,1.04,1.14,...,0.84,1.27,1.08,1.54,1.25,1.02,1.01,0.97,0.85,1.31
2,0.64,0.71,0.96,0.62,0.89,1.16,1.18,1.16,1.04,1.13,...,0.84,1.27,1.08,1.54,1.25,1.02,1.01,0.94,0.97,1.31
3,0.63,0.71,0.95,0.61,0.89,1.16,1.17,1.15,1.03,1.13,...,0.85,1.27,1.08,1.54,1.24,1.02,1.01,0.93,0.96,1.31
4,0.61,0.71,0.94,0.60,0.89,1.16,1.17,1.16,1.03,1.12,...,0.85,1.27,1.08,1.54,1.24,1.02,1.01,0.92,0.95,1.31


In [45]:
print(phreatic_level_lots_nodes_df.shape)
phreatic_level_lots_nodes_df.isnull().values.any()

(4880, 25)


False

In [46]:
# We export it 
phreatic_level_lots_nodes_df.to_csv( '../../../../data/interim/PhreaticLevel/'  \
                    '28-weeks_January-12_August-03_2018/' \
                    'PORV_all_lots_nodes_columns_without_scaling.csv', sep=',', header=True, index=False)

## 1.4 Generating descriptive data to dataset
---

In [75]:
phreatic_level_lots_nodes_describe = phreatic_level_lots_nodes_df.describe(include='all')

In [76]:
phreatic_level_lots_nodes_describe

,PORVL2N1,PORVL2N2,PORVL4N1,PORVL5N1,PORVL6N1,PORVL7N1,PORVL8N1,PORVL9N1,PORVL10N1,PORVL13N1,...,PORVL18N2,PORVL18N3,PORVL18N4,PORVL21N1,PORVL21N2,PORVL21N3,PORVL21N4,PORVL21N5,PORVL24N1,PORVL24N2
count,4880.000000,4880.000000,4880.000000,4880.000000,4880.000000,4880.000000,4880.000000,4880.00000,4880.000000,4880.00000,...,4880.000000,4880.000000,4880.000000,4880.000000,4880.000000,4880.000000,4880.000000,4880.000000,4880.000000,4880.000000
mean,0.893425,0.992494,1.110967,0.941556,1.340499,1.297390,1.297672,1.23772,1.214489,1.21375,...,1.140096,1.384165,1.376002,1.356663,1.243068,1.200952,1.267708,1.273216,1.099189,1.405167
std,0.345951,0.363783,0.322341,0.434695,0.298315,0.357553,0.316150,0.37050,0.341489,0.34560,...,0.406494,0.284760,0.257117,0.297310,0.356607,0.412263,0.379211,0.281943,0.452988,0.294662
min,0.060000,0.060000,0.090000,0.060000,0.190000,0.120000,0.120000,0.06000,0.100000,0.08000,...,0.070000,0.290000,0.620000,0.360000,0.090000,0.060000,0.220000,0.150000,0.140000,0.370000
25%,0.710000,0.750000,0.910000,0.590000,1.190000,1.090000,1.100000,0.93000,0.930000,0.96000,...,0.840000,1.210000,1.180000,1.080000,0.950000,0.860000,0.960000,1.090000,0.760000,1.210000
50%,0.840000,0.930000,1.020000,0.790000,1.370000,1.270000,1.220000,1.14000,1.120000,1.14000,...,1.080000,1.410000,1.360000,1.280000,1.140000,1.060000,1.220000,1.290000,1.060000,1.390000
75%,1.110000,1.390000,1.390000,1.340000,1.650000,1.700000,1.670000,1.70000,1.540000,1.51000,...,1.560000,1.700000,1.650000,1.700000,1.700000,1.680000,1.700000,1.460000,1.580000,1.700000
max,1.700000,1.700000,1.700000,1.700000,1.680000,1.700000,1.700000,1.70000,1.700000,1.70000,...,1.700000,1.700000,1.700000,1.700000,1.700000,1.700000,1.700000,1.700000,1.700000,1.700000


In [77]:
# Export this descriptive data to comma separated values and java script object notation
phreatic_level_lots_nodes_describe.to_csv('../../../../data/interim/PhreaticLevel/28-weeks_January-12_August-03_2018/' +'\n' 
                              'Phreatic_Level_Describe_January-12_August-03.cvs', sep=',', header=True, index=True)
phreatic_level_lots_nodes_describe.to_json('../../../../data/interim/PhreaticLevel/28-weeks_January-12_August-03_2018/' +'\n'
                                     'Phreatic_Level_Describe_January-12_August-03.json')

## 2. Creating Phreatic Level Training and Testing datasets

We have a **`phreatic_level_lots_nodes_df`** dataset with 4880 samples rows.


In [78]:
print(phreatic_level_lots_nodes_df.shape)
phreatic_level_lots_nodes_df.head()

(4880, 25)


,PORVL2N1,PORVL2N2,PORVL4N1,PORVL5N1,PORVL6N1,PORVL7N1,PORVL8N1,PORVL9N1,PORVL10N1,PORVL13N1,...,PORVL18N2,PORVL18N3,PORVL18N4,PORVL21N1,PORVL21N2,PORVL21N3,PORVL21N4,PORVL21N5,PORVL24N1,PORVL24N2
0,0.65,0.71,0.96,0.62,0.90,1.16,1.19,1.16,1.05,1.14,...,0.84,1.27,1.08,1.55,1.25,1.01,1.01,0.96,0.86,1.31
1,0.65,0.71,0.96,0.63,0.89,1.16,1.19,1.16,1.04,1.14,...,0.84,1.27,1.08,1.54,1.25,1.02,1.01,0.97,0.85,1.31
2,0.64,0.71,0.96,0.62,0.89,1.16,1.18,1.16,1.04,1.13,...,0.84,1.27,1.08,1.54,1.25,1.02,1.01,0.94,0.97,1.31
3,0.63,0.71,0.95,0.61,0.89,1.16,1.17,1.15,1.03,1.13,...,0.85,1.27,1.08,1.54,1.24,1.02,1.01,0.93,0.96,1.31
4,0.61,0.71,0.94,0.60,0.89,1.16,1.17,1.16,1.03,1.12,...,0.85,1.27,1.08,1.54,1.24,1.02,1.01,0.92,0.95,1.31


We'll divide it into two differents datasets:

- Training dataset
- Testing dataset

This is executed through [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html  "sklearn.model_selection.train_test_split") function of this way:

`train_test_split` receive as a data parameter a numpy array, we have to turn the 

**`phreatic_level_lots_nodes_df`** dataframe to numpy array such as follow:

In [79]:
# numpy_phreatic_level_lots_nodes = phreatic_level_lots_nodes_df.reset_index().values
numpy_phreatic_level_lots_nodes = phreatic_level_lots_nodes_df.values

In [80]:
# My numpy_phreatic_level_lots_nodes variable now is a numpy array
print(numpy_phreatic_level_lots_nodes.shape)
print(type(numpy_phreatic_level_lots_nodes))
numpy_phreatic_level_lots_nodes

(4880, 25)
<class 'numpy.ndarray'>


array([[0.65      , 0.71      , 0.96      , ..., 0.96      , 0.86      ,
        1.31      ],
       [0.65      , 0.71      , 0.96      , ..., 0.97      , 0.85      ,
        1.31      ],
       [0.64      , 0.71      , 0.96      , ..., 0.94      , 0.97      ,
        1.31      ],
       ...,
       [0.82069245, 0.80241544, 0.94910767, ..., 1.19825056, 0.9261183 ,
        1.16      ],
       [0.82069245, 0.80241544, 0.94910767, ..., 1.19825056, 0.9261183 ,
        1.17      ],
       [0.82069245, 0.80241544, 0.94910767, ..., 1.19825056, 0.9261183 ,
        1.27451648]])

We compose the following datasets from **`numpy_phreatic_level_lots_nodes`** array :

- `numpy_phreatic_level_lots_nodes_train`, which is the training matrix
- `numpy_phreatic_level_lots_nodes_test`, the testing matrix

We use the [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html  "sklearn.model_selection.train_test_split") function to create the training and testing dataset. Their parameters are:

![alt text](https://cldup.com/hukPWvvLxt-3000x3000.png "klearn.model_selection.train_test_split")


- The first parameter **should be an array**, then we pass the **`numpy_phreatic_level_lots_nodes`** which contain all column features of luminosity.

Of this way, we pass all data (9752 samples rows) to from them `numpy_phreatic_level_lots_nodes_train`

and `numpy_phreatic_level_lots_nodes_test` will be created

- The `test_size=0.5` parameter means a 50% division; which means 
that half of the data goes to the test dataset and the other half 
goes to the training dataset.

A good choice for the size of tests **is usually 0.2 ie 20% or 0.25 or even 30%.** 

In some rare cases we will have 40% but almost never 0.5 or 50%

We choose 20% which means that we will have 20% of 9752 samples or observations for the test data set, 

in this case **4880 * 0.2 = 976 samples or records for the test data set**


- The `train_size` parameter is the training dataset size.  **`test_size + train_size = 1 or 100%`**, 
then isn't necessary include it, because wheter we include to `test_size = 0.2`, then the remaining 
data will be to `train_size` this means **0.8 or 80%**

This means that **4880 * 0.8 = 3904  training dataset samples rows. **


- random_state is a seed or data source for generating random values for the data sets. 

If this parameter is not passed, the data will be generated in a random way, but in the way as by default numpy works them.

In [81]:
numpy_phreatic_level_lots_nodes_train, numpy_phreatic_level_lots_nodes_test = train_test_split(numpy_phreatic_level_lots_nodes, test_size = 0.2, random_state = 0)

In [82]:
# We have 3904 rows to luminosity_luxes_train
print(type(numpy_phreatic_level_lots_nodes_train))
print("The dimensionality of wind_direction training dataset is: " +'\n' , numpy_phreatic_level_lots_nodes_train.shape)
print('\n')

# And we have 976 rows to luminosity_luxes_test
print(type(numpy_phreatic_level_lots_nodes_test))
print("The dimensionality of wind_direction testing dataset is: " +'\n' , numpy_phreatic_level_lots_nodes_test.shape)

<class 'numpy.ndarray'>
The dimensionality of wind_direction training dataset is: 
 (3904, 25)


<class 'numpy.ndarray'>
The dimensionality of wind_direction testing dataset is: 
 (976, 25)


In this way, the model, as you progressively learn the correlations in the training set, the better the prediction of the results in the test set.

But if the model learns too much from memory the correlations of the training sets, 

that is to say, when one learns from memory and does not understand things, then it 

will have problems to predict what is happening on the set of tests, because it is 

learned for difficult correlations, if the logic is not well understood and you can 

not make good predictions. This is called overfitting or overfitting


The really important thing is to understand that we need to have two different datasets

- Training set with which the ML model learns
- Test set, on which we test whether the ML model correctly learned the correlations

---
#  3. Feature Scaling Luminosity training and testing dataset  
---

[This article post](http://benalexkeen.com/feature-scaling-with-scikit-learn/  "Feature Scaling with scikit-learn") it's a great reference to explore the features scaling methods
on scikit learn

- `StandardScaler` assume that data is normally distributed at the level of each characteristic or variable. If the data is not normally distributed, it is not the best alternative to use for scaling.  

- `Min-Max Scaler` it is probably the most famous scaling algorithm and what it does is resize the range to leave it in a dimension of 0 to 1 or -1 and 1 (in case there are negative values in the original dataset of input data)

This scale of maximums and minimums works best for cases where standard scaling may not work properly. If the distribution is not Gaussian or the standard deviation is very small the escalation of maximums and minimums is the best idea.
However, it is sensitive to outliers or outliers, so if there are outliers in the data it is better to consider robust scaling.

- `Robust Scaler` it is similar to the previous one of maximums and minimums, only that it uses interquartile ranges instead of maximums and minimums, which makes it robust for the outliers

- `Normalizer` which scales each value, dividing each value by its magnitude in n dimensional spaces for n number of characteristics.


---

We will use the scaling of maximums and minimums to scale the water table data, because the standard deviation is very small, 

it does not have atypical values and it does not follow a normal distribution (you have to check this)

We apply the maximum and minimum scaling. We provide a rank or base scale that **will be between 0 and 1** using an object,

[MinMaxScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html "sklearn.preprocessing.MinMaxScaler") which transforms each characteristic, (in this 

case it will be the columns feature of `PORVL.*`) individually according to a given range.

Product of its applicability, generates these attributes in the dataset, already transformed: 

![alt text](https://cldup.com/lTIv4HXgTk-3000x3000.png "sklearn.preprocessing.MinMaxScaler")

## 3.1 We apply maximium and minimum feature scaling to Phreatic level training dataset

In [83]:
# We provide a base scale range
scaler = MinMaxScaler(feature_range=(0, 1))

print("Remember our phreatic level training data " + '\n', numpy_phreatic_level_lots_nodes_train)

Remember our phreatic level training data 
 [[0.85 1.39 1.39 ... 1.43 1.54 1.7 ]
 [0.7  1.39 1.2  ... 1.43 1.21 1.7 ]
 [0.56 0.91 0.96 ... 1.01 0.98 1.33]
 ...
 [1.44 1.52 1.59 ... 1.58 1.7  1.7 ]
 [0.75 0.64 0.88 ... 1.12 0.6  1.21]
 [0.48 0.43 0.73 ... 1.01 0.43 1.05]]


With the [fit](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler.fit  "MinMaxScaler.fit")
we compute the maximum and minimum value of  `numpy_phreatic_level_lots_nodes_train` dataset to be used in the subsequent scaling 

We assing these values to `minmax_scale_training` variable.

In [84]:
minmax_scale_training = scaler.fit(numpy_phreatic_level_lots_nodes_train.astype(float))
# print(minmax_scale_training.data_max_)
# http://terrapinssky.blogspot.com/2017/10/pythonresolved-dataconversionwarning.html

Then, we apply the [transform](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler.transform "MinMaxScaler.transform") method to transform these data to maximum and mínimum scale value. 

Here, with this process, the `numpy_phreatic_level_lots_nodes_train` data are scaled between **0 to 1**  selected range 

In [85]:
# transform luminosity_luxes_train data to maximum and mínimum scale value. 
phreatic_level_lots_nodes_minmax_training = minmax_scale_training.transform(numpy_phreatic_level_lots_nodes_train)

In [86]:
print ("And, these are our scaled data: " + '\n')
phreatic_level_lots_nodes_minmax_training

And, these are our scaled data: 



array([[0.48170732, 0.81097561, 0.80503145, ..., 0.82580645, 0.8974359 ,
        1.        ],
       [0.3902439 , 0.81097561, 0.68553459, ..., 0.82580645, 0.68589744,
        1.        ],
       [0.30487805, 0.51829268, 0.53459119, ..., 0.55483871, 0.53846154,
        0.72180451],
       ...,
       [0.84146341, 0.8902439 , 0.93081761, ..., 0.92258065, 1.        ,
        1.        ],
       [0.42073171, 0.35365854, 0.48427673, ..., 0.62580645, 0.29487179,
        0.63157895],
       [0.25609756, 0.22560976, 0.38993711, ..., 0.55483871, 0.18589744,
        0.5112782 ]])

In [87]:
print('Luminosity Training dataset. Minimum value after MaxMinScaler:\nLuminosity={:.1f}'
      .format(phreatic_level_lots_nodes_minmax_training[:,0].min()))

print('Luminosity Training dataset. Maximum value after MaxMinScaler:\nLuminosity={:.1f}'
      .format(phreatic_level_lots_nodes_minmax_training[:,0].max()))

Luminosity Training dataset. Minimum value after MaxMinScaler:
Luminosity=0.0
Luminosity Training dataset. Maximum value after MaxMinScaler:
Luminosity=1.0


Then, our  MinMaxScaler normalized training dataset is `phreatic_level_lots_nodes_minmax_training` numpy array

- We export this array to comma separated values 

In [88]:
phreatic_level_lots_nodes_train_df = pd.DataFrame(phreatic_level_lots_nodes_minmax_training, columns=cols)

In [89]:
print(phreatic_level_lots_nodes_train_df.shape)
phreatic_level_lots_nodes_train_df.head()

(3904, 25)


,PORVL2N1,PORVL2N2,PORVL4N1,PORVL5N1,PORVL6N1,PORVL7N1,PORVL8N1,PORVL9N1,PORVL10N1,PORVL13N1,...,PORVL18N2,PORVL18N3,PORVL18N4,PORVL21N1,PORVL21N2,PORVL21N3,PORVL21N4,PORVL21N5,PORVL24N1,PORVL24N2
0,0.481707,0.810976,0.805031,0.780488,0.743056,1.000000,0.936709,1.000000,1.00000,0.827160,...,0.914110,0.971631,1.000000,1.000000,1.000000,1.000000,1.000000,0.825806,0.897436,1.000000
1,0.390244,0.810976,0.685535,0.609756,0.618056,0.816456,0.816456,0.853659,1.00000,0.746914,...,0.815951,0.829787,0.685185,0.992537,0.949686,1.000000,0.891892,0.825806,0.685897,1.000000
2,0.304878,0.518293,0.534591,0.329268,0.451389,0.626582,0.645570,0.664634,0.56250,0.604938,...,0.484663,0.709220,0.453704,0.895522,0.685535,0.591463,0.547297,0.554839,0.538462,0.721805
3,0.030488,0.000000,0.264151,0.128049,0.173611,0.265823,0.348101,0.439024,0.37500,0.296296,...,0.190184,0.460993,0.231481,0.589552,0.477987,0.378049,0.229730,0.335484,0.038462,0.308271
4,0.298780,0.359756,0.427673,0.262195,0.840278,0.658228,0.658228,0.481707,0.53125,0.518519,...,0.478528,0.780142,0.564815,0.492537,0.421384,0.420732,0.547297,0.567742,0.391026,0.661654


In [90]:
phreatic_level_lots_nodes_train_df[phreatic_level_lots_nodes_train_df['PORVL2N1']>1]

,PORVL2N1,PORVL2N2,PORVL4N1,PORVL5N1,PORVL6N1,PORVL7N1,PORVL8N1,PORVL9N1,PORVL10N1,PORVL13N1,...,PORVL18N2,PORVL18N3,PORVL18N4,PORVL21N1,PORVL21N2,PORVL21N3,PORVL21N4,PORVL21N5,PORVL24N1,PORVL24N2


In this way we have the dataset `phreatic_level_lots_nodes_train_df` standardized and training, and export it to a .csv file

In [91]:
phreatic_level_lots_nodes_train_df.to_csv('../../../../data/processed/PhreaticLevel/28-weeks_January-12_August-03_2018/' +'\n' 
                                 'Phreatic-Level_Normalized_TRAINING_January-12_August-03.csv', sep=',', header=True, index=False)

## 3.2 We apply maximium and minimum feature scaling to phreatic level testing dataset

In [92]:
print("Remember our phreatic level testing data " + '\n', numpy_phreatic_level_lots_nodes_test)

Remember our phreatic level testing data 
 [[0.79 0.71 0.89 ... 1.24 0.77 1.01]
 [0.75 0.66 0.81 ... 1.19 0.55 1.4 ]
 [0.88 1.39 1.41 ... 1.42 1.62 1.7 ]
 ...
 [0.72 1.02 1.05 ... 1.23 0.92 1.48]
 [0.93 0.83 1.04 ... 1.35 1.05 1.35]
 [0.47 0.53 0.84 ... 1.4  0.99 1.35]]


In [93]:
minmax_scale_test = scaler.fit(numpy_phreatic_level_lots_nodes_test.astype(float))
# transform phreatic level data to maximum and mínimum scale value. 
phreatic_level_lots_nodes_minmax_test = minmax_scale_test.transform(numpy_phreatic_level_lots_nodes_test)

In [94]:
print ("And, these are our testing scaled data: " + '\n')
phreatic_level_lots_nodes_minmax_test

And, these are our testing scaled data: 



array([[0.43478261, 0.39634146, 0.49689441, ..., 0.65671642, 0.40384615,
        0.448     ],
       [0.40993789, 0.36585366, 0.44720497, ..., 0.61940299, 0.26282051,
        0.76      ],
       [0.49068323, 0.81097561, 0.81987578, ..., 0.79104478, 0.94871795,
        1.        ],
       ...,
       [0.39130435, 0.58536585, 0.59627329, ..., 0.64925373, 0.5       ,
        0.824     ],
       [0.52173913, 0.4695122 , 0.59006211, ..., 0.73880597, 0.58333333,
        0.72      ],
       [0.23602484, 0.28658537, 0.46583851, ..., 0.7761194 , 0.54487179,
        0.72      ]])

In [95]:
print('Phreatic Level Testing dataset. Minimum value after MaxMinScaler:\nLuminosity={:.1f}'
      .format(phreatic_level_lots_nodes_minmax_test[:,0].min()))

print('Phreatic Level Testing dataset. Maximum value after MaxMinScaler:\nLuminosity={:.1f}'
      .format(phreatic_level_lots_nodes_minmax_test[:,0].max()))

Phreatic Level Testing dataset. Minimum value after MaxMinScaler:
Luminosity=0.0
Phreatic Level Testing dataset. Maximum value after MaxMinScaler:
Luminosity=1.0


Then, our  MinMaxScaler normalized training dataset is `phreatic_level_lots_nodes_minmax_test` numpy array

- We export this array to comma separated values

In [96]:
phreatic_level_lots_nodes_test_df = pd.DataFrame(phreatic_level_lots_nodes_minmax_test, columns=cols)

In [97]:
print(phreatic_level_lots_nodes_test_df.shape)
phreatic_level_lots_nodes_test_df.head()

(976, 25)


,PORVL2N1,PORVL2N2,PORVL4N1,PORVL5N1,PORVL6N1,PORVL7N1,PORVL8N1,PORVL9N1,PORVL10N1,PORVL13N1,...,PORVL18N2,PORVL18N3,PORVL18N4,PORVL21N1,PORVL21N2,PORVL21N3,PORVL21N4,PORVL21N5,PORVL24N1,PORVL24N2
0,0.434783,0.396341,0.496894,0.31875,1.000000,0.653846,0.608974,0.592357,0.375940,0.480519,...,0.401274,0.355932,0.392523,0.528455,0.527950,0.325758,0.734694,0.656716,0.403846,0.448
1,0.409938,0.365854,0.447205,0.26250,0.744966,0.500000,0.583333,0.388535,0.353383,0.461039,...,0.464968,0.618644,0.485981,0.463415,0.416149,0.212121,0.496599,0.619403,0.262821,0.760
2,0.490683,0.810976,0.819876,0.78125,0.771812,1.000000,0.961538,1.000000,1.000000,0.811688,...,0.904459,1.000000,0.971963,1.000000,1.000000,0.992424,1.000000,0.791045,0.948718,1.000
3,0.285714,0.353659,0.434783,0.25000,0.845638,0.653846,0.647436,0.452229,0.458647,0.506494,...,0.458599,0.737288,0.570093,0.455285,0.422360,0.272727,0.557823,0.514925,0.378205,0.640
4,0.993789,0.871951,0.993789,1.00000,1.000000,1.000000,0.980769,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.983740,0.863354,0.787879,1.000000,0.992537,0.878205,1.000


In [98]:
phreatic_level_lots_nodes_test_df.to_csv('../../../../data/processed/PhreaticLevel/28-weeks_January-12_August-03_2018/' +'\n' 
                                 'Phreatic-Level_Normalized_TESTING_January-12_August-03.csv', sep=',', header=True, index=False)